<a href="https://colab.research.google.com/github/MiguelAngeloTr/BIGDATA/blob/main/C1/Parcial1/Parcial_1_BIGDATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


##Universidad Autónoma de Occidente

##Facultad de Ingenierías y Ciencias Básicas

##Curso: Big Data

##Autor: Sergio A. Cantillo

##Correo: sacantillo@uao.edu.co

---

### **NOMBRE COMPLETO:**

###**CODIGO:**

### **PROGRAMA:**

---

### **Condiciones**

*   Pueden usar los archivos Google Colab trabajados en clase.
*   No pueden usar IA para buscar las respuetas del examen. Quien sea visto en flagrancia, el examen será **anulado**.
*   Se contestan las preguntas 1, 2 y 3 Obligatoriamente, del resto se escogen 2 para hacer 5. Si contestan todas las 6 preguntas correctamente bonifican para el segundo parcial.
*   Pueden usar el material de referencia de UAO virtual
*   Subir el archivo con extensión .ipynb a la plataforma antes de las 12:59 PM.


# **Examen 1**

El conjunto de datos AB_NYC_2019.csv contiene información acerca de los listados de Airbnb en la ciudad de Nueva York. Incluye variables como el nombre del alojamiento, el nombre del anfitrión, la ubicación geográfica (vecindario y coordenadas), el tipo de habitación, el precio por noche, el número mínimo de noches requeridas para una reserva, la disponibilidad durante el año, entre otros detalles relevantes. El objetivo en este análisis de datos consiste en utilizar las herramientas computacionales y estadísticas vistas hasta el momento en el curso que logren contribuir a la caracterización de estos alojamientos



## **1.**   A partir de la carga del conjunto de datos crudo, realice los procesos de adecuación (Ej. tipos de dato) y limpieza de (Ej. identificación y eliminación de columnas) requeridos.

##Responda:

## 1.1 ¿Cual es el tamaño (filas,columnas) del conjunto de datos crudo?, y luego de los procesos solicitados?

## 1.2 Indique brevemente si fueron eliminadas columnas de la base de datos y el criterio correspondiente.

---



## **2.** A partir de la variable precio del alojamiento (price), obtenga los coeficientes de asimetria y forma, ademas grafíquela a través de un histograma y un diagrama de cajas.

## Responda:

## 2.1 De acuerdo a los coeficientes de asimetria y forma, ¿Qué clasificación presenta esta variable? ¿Corresponde a una distribución normal convencional?

---



## **3.**   Realice una tabla donde se determine la distribución de los tipos de alojamiento disponibles (room_type) por vecindario (neighborhood_group), incluyendo el precio promedio para cada uno. Asimismo, Grafique esta distribución con un diagrama de barras donde el eje X sean los vecindarios, el eje Y las cantidades. Ademas que existan barras por cada tipo de alojamiento.

## Responda:

## 3.1 ¿Qué se puede decir acerca de la relación entre el precio y tipo de alojamiento respecto del vecindario donde estan ubicados?

## 3.2 ¿Qué vecindarios presentan la mayor cantidad de alojamientos tipo Private room?

## 3.3 ¿Qué se puede decir de la distribución de los alojamientos en manhattan con respecto a los demas vecindarios?

---



## **4.**   Realice una tabla donde se determine por vecindario (neighbourhood_group) la cantidad de alojamientos totales, su equivalente porcentual y su disponibilidad promedio (availability_365). Ordene los resultados de acuerdo al porcentaje (mayor a menor)

## Responda:

## 4.1. Teniendo en cuenta la información de esta tabla, ¿En cuál vecindario seria más probable encontrar alojamiento disponible?

## 4.2 ¿Qué se puede decir acerca de la relación de los alojamientos totales y la disponibilidad promedio en los vecindarios?

---




## **5.** Construya una matriz de correlación considerando el precio, la disponibilidad, y el numero de noches minimo de alojamiento.
## Responda:
## 5.1 ¿Cómo afecta la disponibilidad (minimum_nights) al precio de los alojamientos en Airbnb NYC?

## **6.** A partir de la variable availability_365, realice una recodificación las siguientes condiciones y añadala a dataframe con el nombre de "availability level":

* Low disp: 0 a 119 días
* Med disp: 120 a 240 días
* High disp: 241 a 360 días

## 6.1 A partir de este nuevo dataframe, entregue una tabla donde por este nivel de disponibilidad se muestre el total de alojamientos. ¿Que se puede decir acerca de esta información?